In [23]:
import os

os.environ['OLLAMA_HOST'] = 'http://192.168.48.92:11434'

In [24]:
import ollama
ollama.list()

ListResponse(models=[Model(model='qwen3-embedding-8b:latest', modified_at=datetime.datetime(2025, 6, 16, 14, 8, 31, 856634, tzinfo=TzInfo(+08:00)), digest='78df73127e45da57b861d27f4a66b9c1e61025ce3d84ab83babb8d7bf0ab246f', size=8047105511, details=ModelDetails(parent_model='', format='gguf', family='qwen3', families=['qwen3'], parameter_size='7.6B', quantization_level='Q8_0'))])

In [69]:
import pandas as pd

file_path = r"C:\Lee\work\contract\csv\v3\contract_data.csv"
df = pd.read_csv(file_path)

df = df.head()

In [73]:
def process_row_embedding(row):
    # 检查各字段是否为 NaN 或空，如果是则替换为空字符串
    project_name = str(row["project_name"]) if pd.notna(row["project_name"]) else ""
    subitem_name = str(row["subitem_name"]) if pd.notna(row["subitem_name"]) else ""
    supplier = str(row["supplier"]) if pd.notna(row["supplier"]) else ""
    
    # 拼接非空字段
    combined_text = project_name + subitem_name + supplier
    
    # 如果所有字段均为空，返回 None 或默认值
    if not combined_text:
        return None  # 或返回一个默认的 embedding 向量
    
    # 调用 embedding 模型
    embedded = ollama.embed(
        model='qwen3-embedding-8b:latest', 
        input=combined_text
    )
    return embedded.embeddings[0]

# df[["meta_vector","other_columns"]] = df.apply(process_row_embedding, axis=1, result_type="expand")
df["meta_vector"] = df.apply(process_row_embedding, axis=1)

In [74]:
df.head()

,contract_meta,doc,contact_no,project_name,subitem_name,total_price_str,total_price,date,supplier,equipment_table,meta_vector
0,"{'buyer_contract_number': 'TGYR-2023-01', 'sel...",买方合同编号：TGYR-2023-01卖方合同编号：订 货 合 同项目名称：山东泰山钢铁集团...,TGYR-2023-01,山东泰山钢铁集团有限公司265㎡烧环冷机余热系统改造工程EPC总承包余热回收项目,环冷机余热锅炉,肆佰伍拾柒万圆整,457.0,2023-08,江苏东九重工股份有限公司,"['<table><thead><tr><th>序号</th><th colspan=""2""...","[-0.013221039, 0.009854255, -0.009858984, 0.01..."
1,"{'buyer_contract_number': 'BGZJQT-E-2019-01', ...",买方合同编号：BGZJQT-E-2019-01卖方合同编号：订 货 合 同项目名称：宝钢湛江...,BGZJQT-E-2019-01,宝钢湛江钢铁有限公司炼铁厂球团烟气脱硝设施球团烟气SCR脱硝系统,回转式烟气换热器（GGH）,肆佰玖拾玖万圆整,4990000.0,2019-10,容客傲华新能源科技（上海）有限公司,['<table><thead><tr><th>序号</th><th>货物名称</th><t...,"[-0.017960778, 0.019593611, -0.008444737, -0.0..."
2,"{'buyer_contract_number': 'SZWF-CG-2011-001', ...","买方合同编号：SZWF-CG-2011-001卖方合同编号：<a id=""OLE_LINK2...",SZWF-CG-2011-001,深圳危险废物焚烧处置工程EPC项目,余热锅炉成套设备供货,人民币贰佰贰拾万元整（￥220.00万元）,220.0,2011-05,长沙锅炉厂有限责任公司,['<table><tr><td>序号</td><td>名称</td><td>金额（万元）<...,"[-0.012383494, 0.008031144, -0.025164675, -0.0..."
3,"{'buyer_contract_number': 'BGYR3-E-2014-01', '...",买方合同编号：BGYR3-E-2014-01卖方合同编号：本钢集团北营公司300㎡、360㎡...,BGYR3-E-2014-01,本钢集团北营公司300㎡、360㎡烧结余热发电工程项目,NaN,玖佰捌拾陆万圆整,986.0,2014-11,南通万达锅炉有限公司,['<table><tr><td>序号</td><td>货物名称</td><td>数 量</...,"[-0.017734123, 0.014578746, -0.014225431, 0.00..."
4,"{'buyer_contract_number': 'WGZJGLD-2019-01', '...",买方合同编号：WGZJGLD-2019-01卖方合同编号：订 货 合 同项目名称：舞钢中加钢...,WGZJGLD-2019-01,舞钢中加钢铁有限公司122万吨生产线 配套180㎡烧结机项目工程,余热锅炉,肆佰伍拾伍万元,455.0,2019-05,南京圣诺热管有限公司,['<table><thead><tr><th>序号</th><th>货物名称</th><t...,"[-0.009228017, 0.027426142, -0.0048992485, 0.0..."


# Load db

In [94]:
import lancedb
import pandas as pd
import pyarrow as pa

db = lancedb.connect(r"C:\Lee\work\contract\csv\v3\contract_full_lancedb")  # 指定数据库目录，不存在会自动创建

schema = pa.schema([
    pa.field("meta_vector", pa.list_(pa.float32(),4096)),
    pa.field("doc", pa.utf8()),
    pa.field("contract_meta", pa.utf8()),
    pa.field("contact_no", pa.utf8()),
    pa.field("project_name", pa.utf8()),
    pa.field("subitem_name", pa.utf8()),
    pa.field("total_price", pa.float32()),
    pa.field("total_price_str", pa.utf8()),
    pa.field("date", pa.utf8()),
    pa.field("supplier", pa.utf8()),
    pa.field("equipment_table", pa.utf8()),
    pa.field("equipments_vector", pa.list_(pa.float32(),4096)),
])

table = db.create_table("contract_table", schema=schema,mode="overwrite")

df['total_price'] = pd.to_numeric(df['total_price'], errors='coerce').astype('float32')
# 其它字段转为字符串
str_columns = [
    'contract_meta','doc', 'contact_no', 'project_name', 'subitem_name',
    'total_price_str', 'supplier', 'equipment_table', 'date'
]
for col in str_columns:
    df[col] = df[col].astype(str)
    
dict_list = df.to_dict(orient='records')

table.add(dict_list)

AddResult(version=2)

# Check Data Ollama（本地embedding模型有问题，语义搜索基本失效）

In [76]:
db = lancedb.connect(r"C:\Lee\work\contract\csv\v3\contract_full_lancedb") 
table = db.open_table("contract_table")

In [89]:
embedded = ollama.embed(model='qwen3-embedding-8b:latest', input="深圳危险废物焚烧处置工程EPC项目余热锅炉成套设备供货")
embedding = embedded.embeddings[0]

In [90]:
search_vector_results = table.search(embedding,vector_column_name="meta_vector").limit(10).to_pandas()
search_vector_results

,meta_vector,doc,contract_meta,contact_no,project_name,subitem_name,total_price,total_price_str,date,supplier,equipment_table,equipments_vector,_distance
0,"[-0.017734123, 0.014578746, -0.014225431, 0.00...",买方合同编号：BGYR3-E-2014-01卖方合同编号：本钢集团北营公司300㎡、360㎡...,"{'buyer_contract_number': 'BGYR3-E-2014-01', '...",BGYR3-E-2014-01,本钢集团北营公司300㎡、360㎡烧结余热发电工程项目,nan,986.0,玖佰捌拾陆万圆整,2014-11,南通万达锅炉有限公司,['<table><tr><td>序号</td><td>货物名称</td><td>数 量</...,None,1.113765
1,"[-0.013221039, 0.009854255, -0.009858984, 0.01...",买方合同编号：TGYR-2023-01卖方合同编号：订 货 合 同项目名称：山东泰山钢铁集团...,"{'buyer_contract_number': 'TGYR-2023-01', 'sel...",TGYR-2023-01,山东泰山钢铁集团有限公司265㎡烧环冷机余热系统改造工程EPC总承包余热回收项目,环冷机余热锅炉,457.0,肆佰伍拾柒万圆整,2023-08,江苏东九重工股份有限公司,"['<table><thead><tr><th>序号</th><th colspan=""2""...",None,1.201542
2,"[-0.012383494, 0.008031144, -0.025164675, -0.0...","买方合同编号：SZWF-CG-2011-001卖方合同编号：<a id=""OLE_LINK2...","{'buyer_contract_number': 'SZWF-CG-2011-001', ...",SZWF-CG-2011-001,深圳危险废物焚烧处置工程EPC项目,余热锅炉成套设备供货,220.0,人民币贰佰贰拾万元整（￥220.00万元）,2011-05,长沙锅炉厂有限责任公司,['<table><tr><td>序号</td><td>名称</td><td>金额（万元）<...,None,1.222081
3,"[-0.009228017, 0.027426142, -0.0048992485, 0.0...",买方合同编号：WGZJGLD-2019-01卖方合同编号：订 货 合 同项目名称：舞钢中加钢...,"{'buyer_contract_number': 'WGZJGLD-2019-01', '...",WGZJGLD-2019-01,舞钢中加钢铁有限公司122万吨生产线 配套180㎡烧结机项目工程,余热锅炉,455.0,肆佰伍拾伍万元,2019-05,南京圣诺热管有限公司,['<table><thead><tr><th>序号</th><th>货物名称</th><t...,None,1.222424
4,"[-0.017960778, 0.019593611, -0.008444737, -0.0...",买方合同编号：BGZJQT-E-2019-01卖方合同编号：订 货 合 同项目名称：宝钢湛江...,"{'buyer_contract_number': 'BGZJQT-E-2019-01', ...",BGZJQT-E-2019-01,宝钢湛江钢铁有限公司炼铁厂球团烟气脱硝设施球团烟气SCR脱硝系统,回转式烟气换热器（GGH）,4990000.0,肆佰玖拾玖万圆整,2019-10,容客傲华新能源科技（上海）有限公司,['<table><thead><tr><th>序号</th><th>货物名称</th><t...,None,1.234487


In [79]:
search_vector_results.loc[2,'contract_meta']

"{'buyer_contract_number': 'TGYR-2023-01', 'seller_contract_number': None, 'contract_type': '订货合同', 'project_name': '山东泰山钢铁集团有限公司265㎡烧环冷机余热系统改造工程EPC总承包余热回收项目', 'sub_project_name': '环冷机余热锅炉', 'buyer': {'name': '湖南中冶长天节能环保技术有限公司', 'address': '长沙市岳麓区节庆路7号', 'contact_person': '戴亮', 'phone': '18773083301', 'bank_account': '浦发银行长沙东塘支行 6605 0155 2000 0246 8', 'tax_id': '9143 0100 5849 3952 00'}, 'seller': {'name': '江苏东九重工股份有限公司', 'address': '盐城高新区华锐南路9号', 'contact_person': '刘忠', 'phone': '0515-88597534', 'bank_account': '盐城市中行亭湖支行 5456 5820 7770', 'tax_id': '91320900140334109F'}, 'total_amount': 457.0, 'total_amount_str': '肆佰伍拾柒万圆整', 'tax_excluded_amount': None, 'tax_amount': None, 'signing_date': '2023-08'}"

# Vectorize use AliBaba text v4（切换线上v4模型）

In [117]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
api_key=os.getenv("QWEN_API_KEY")
base_url=os.getenv("QWEN_API_BASE_URL")
embedding_model_id = 'text-embedding-v4'

def get_embedding(text,model='text-embedding-v4',dimensions=2048):
    client = OpenAI(
        api_key=api_key, 
        base_url=base_url
    )

    completion = client.embeddings.create(
        model=model,
        input=text,
        dimensions=dimensions, # 指定向量维度（仅 text-embedding-v3及 text-embedding-v4支持该参数）
        encoding_format="float"
    )
    
    return completion.data[0].embedding

In [118]:
import pandas as pd

file_path = r"C:\Lee\work\contract\csv\v3\contract_data.csv"
df = pd.read_csv(file_path)

# df = df.head() # for testing

In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 756 entries, 0 to 755
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   contract_meta    756 non-null    object 
 1   doc              756 non-null    object 
 2   contact_no       725 non-null    object 
 3   project_name     723 non-null    object 
 4   subitem_name     626 non-null    object 
 5   total_price_str  754 non-null    object 
 6   total_price      754 non-null    float64
 7   date             748 non-null    object 
 8   supplier         753 non-null    object 
 9   equipment_table  756 non-null    object 
dtypes: float64(1), object(9)
memory usage: 59.2+ KB


In [120]:
def process_row_embedding(row):
    # 检查各字段是否为 NaN 或空，如果是则替换为空字符串
    project_name = str(row["project_name"]) if pd.notna(row["project_name"]) else ""
    subitem_name = str(row["subitem_name"]) if pd.notna(row["subitem_name"]) else ""
    supplier = str(row["supplier"]) if pd.notna(row["supplier"]) else ""
    
    # 拼接非空字段
    combined_text = project_name + subitem_name + supplier
    
    # 如果所有字段均为空，返回 None 或默认值
    if not combined_text:
        return None  # 或返回一个默认的 embedding 向量
    
    # 调用 embedding 模型
    embedding = get_embedding(combined_text)
    return embedding

# df[["meta_vector","other_columns"]] = df.apply(process_row_embedding, axis=1, result_type="expand")
df["meta_vector"] = df.apply(process_row_embedding, axis=1)

In [121]:
df.to_csv(r"C:\Lee\work\contract\csv\v3\contract_data_vectorized.csv", index=False)

In [122]:
import lancedb
import pandas as pd
import pyarrow as pa

db = lancedb.connect(r"C:\Lee\work\contract\csv\v3\contract_full_lancedb")  # 指定数据库目录，不存在会自动创建

schema = pa.schema([
    pa.field("meta_vector", pa.list_(pa.float32(),2048)),
    pa.field("doc", pa.utf8()),
    pa.field("contract_meta", pa.utf8()),
    pa.field("contact_no", pa.utf8()),
    pa.field("project_name", pa.utf8()),
    pa.field("subitem_name", pa.utf8()),
    pa.field("total_price", pa.float32()),
    pa.field("total_price_str", pa.utf8()),
    pa.field("date", pa.utf8()),
    pa.field("supplier", pa.utf8()),
    pa.field("equipment_table", pa.utf8()),
])

table = db.create_table("contract_table", schema=schema,mode="overwrite")

df['total_price'] = pd.to_numeric(df['total_price'], errors='coerce').astype('float32')
# 其它字段转为字符串
str_columns = [
    'contract_meta','doc', 'contact_no', 'project_name', 'subitem_name',
    'total_price_str', 'supplier', 'equipment_table', 'date'
]
for col in str_columns:
    df[col] = df[col].astype(str)
    
dict_list = df.to_dict(orient='records')

table.add(dict_list)

AddResult(version=2)

In [126]:
embedding = get_embedding("宝钢湛江")
search_vector_results = table.search(embedding,vector_column_name="meta_vector").limit(10).to_pandas()
search_vector_results

,meta_vector,doc,contract_meta,contact_no,project_name,subitem_name,total_price,total_price_str,date,supplier,equipment_table,_distance
0,"[0.0043713786, -0.05306807, 0.02987152, 0.0035...",买方合同编号：ZJJXTGH-2021-01卖方合同编号：21CBZJ01订 货 合 同项目...,"{'buyer_contract_number': 'ZJJXTGH-2021-01', '...",ZJJXTGH-2021-01,宝钢湛江解析塔更换项目,解析塔,1.300000e+03,壹仟叁佰万元整,2021-09,湖南中冶长天重工科技有限公司,['<table><thead><tr><th>序号</th><th>货物名称</th><t...,0.600026
1,"[-0.013344941, -0.042148463, 0.018121352, 0.02...",补\t充\t协\t议一.\t合同方买方：\t（以下称：“甲方”）名称：湖南中冶长天节能环保技...,"{'buyer_contract_number': None, 'seller_contra...",nan,宝钢湛江解析塔项目,nan,1.610000e+04,壹万陆仟壹佰元整,nan,巨人通力电梯有限公司,[],0.621815
2,"[0.027805625, -0.02834554, 0.024048716, 0.0167...",宝钢湛江汽水阀门补充合同合同号：ZJYR-E-2014-15A 汽水阀门 ...,"{'buyer_contract_number': 'ZJYR-E-2014-15A', '...",ZJYR-E-2014-15A,宝钢湛江汽水阀门补充合同,nan,6.816600e+00,陆万捌仟壹佰陆拾陆 圆整,2015-08,江西赛迪科技有限公司,"['<table><tr><td colspan=""10"">一．合同内容 </td></...",0.654979
3,"[0.024518514, -0.015139908, 0.04435322, 0.0296...",宝钢湛江手持式对讲机订货合同合同号：ZJYR-E-2014-30 手持式对讲机 ...,"{'buyer_contract_number': 'ZJYR-E-2014-30', 's...",ZJYR-E-2014-30,宝钢湛江手持式对讲机订货合同,nan,1.105650e+04,壹万壹仟零伍拾陆 圆伍角,2015-09,上海佰能工程技术有限公司,"['<table><tr><td colspan=""7"">一．合同内容 </td></t...",0.664037
4,"[0.055063188, -0.042455066, 0.052116383, 0.033...",买方合同编号：BGZJYQSCR-E-2019-14卖方合同编号：订 货 合 同项目名称：宝...,{'buyer_contract_number': 'BGZJYQSCR-E-2019-14...,BGZJYQSCR-E-2019-14,宝钢湛江一、二烧烟气SCR,SCR反应器本体,7.308000e+06,柒佰叁拾万零捌仟,2020-05,上海弘韬建设发展有限公司,['<table><thead><tr><th>序号</th><th>货物名称</th><t...,0.672970
5,"[0.043892294, -0.033288863, 0.06188856, 0.0273...",买方合同编号：BGZJYR-E-2019-03补卖方合同编号：S-30-2019-0080 ...,"{'buyer_contract_number': 'BGZJYR-E-2019-03补',...",BGZJYR-E-2019-03补,宝钢湛江三高炉烧结余热发电,汽轮发电机组,6.800000e+01,陆拾捌万元整（￥680000）,2020-07,中国长江动力集团有限公司,['<table><tr><td>序号</td><td>产品名称</td><td>数量</t...,0.691118
6,"[0.016310306, -0.037951607, 0.048613597, 0.017...",买方合同编号：ZJYR-E-2014-13卖方合同编号：订 货 合 同项目名称：宝钢广东湛江...,"{'buyer_contract_number': 'ZJYR-E-2014-13', 's...",ZJYR-E-2014-13,宝钢广东湛江钢铁基地项目烧结工程余热发电系统,起重设备,6.500000e+01,陆拾伍万圆整,2014-09,上海宝松重型机械工程（集团）有限公司,['<table><tr><td>序号</td><td>货物名称</td><td>型号规格<...,0.698650
7,"[0.030351931, -0.024470588, 0.04789094, 0.0239...",宝钢广东湛江钢铁基地项目烧结工程余热发电系统补充采购合同合同号：ZJYR-E-2014-33...,"{'buyer_contract_number': 'ZJYR-E-2014-33', 's...",ZJYR-E-2014-33,宝钢广东湛江钢铁基地项目烧结工程余热发电系统,nan,1.600000e-01,壹仟陆佰 圆整,2015-10,nan,"['<table><tr><td colspan=""8"">一．合同内容 </td></t...",0.701970
8,"[0.050331485, -0.03738017, 0.03465593, 0.01767...",买方合同编号：BGZJZS-E-2020-11(BC)卖方合同编号：订 货 合 同项目名称：...,{'buyer_contract_number': 'BGZJZS-E-2020-11(BC...,BGZJZS-E-2020-11(BC),宝钢湛江SCR、CO催化氧化中试,汽水阀门补充合同,2.500000e-01,贰仟伍佰 元整,2020-12,双恒阀门集团有限公司,['<table><tr><td>序号</td><td>货物名称</td><td>规格型号<...,0.702136
9,"[0.04157044, -0.033523183, 0.061879177, 0.0261...",买方合同编号：BGZJYR-E-2019-03卖方合同编号：S-30-2019-0080订 ...,"{'buyer_contract_number': 'BGZJYR-E-2019-03', ...",BGZJYR-E-2019-03,宝钢湛江三高炉烧结余热发电,汽轮机组,7.700000e+06,柒佰柒拾万元整,2019-10,中国长江动力集团有限公司,['<table><thead><tr><th>序号</th><th>货物名称</th><t...,0.703366
